<p>Credit to http://www.gadgetsappshacks.com/2015/09/how-to-draw-travel-time-boundary-on.html for part 2</p>

In [1]:
import gmaps
import gmaps.datasets
from datetime import datetime
import requests, json 
import math as Math

In [2]:
#Store API Key to variable 'key'
with open('API Key.txt') as f:
    key = f.readline()
    f.close

In [3]:
#Configure with API Key and set variable 'now' to current time
gmaps.configure(api_key='XXXXXXX')
now = datetime.now()

In [4]:
#Sample map
new_york_coordinates = (40.75, -74.00)
#gmaps.figure(center=new_york_coordinates, zoom_level=12)

In [5]:
#Sample map displaying route
#Define location 1 and 2
Durango = (37.2753,-107.880067)
SF = (37.7749,-122.419416)
#Create the map
fig = gmaps.figure()
#create the layer
layer = gmaps.directions.Directions(Durango, SF,mode='car')
#Add the layer
fig.add_layer(layer)
#fig

In [6]:
#Sample map displaying route (with extra options)
# Latitude-longitude pairs
geneva = (46.2, 6.1)
montreux = (46.4, 6.9)
zurich = (47.4, 8.5)
fig = gmaps.figure()
geneva2zurich = gmaps.directions_layer(
    geneva, zurich, travel_mode="BICYCLING"
)
fig.add_layer(geneva2zurich)
#fig

In [7]:
#Converts given travel time to an integer
def travelTimeInt(travelTime):
    if type(travelTime) is not str:
        pass
    elif "hour" in travelTime:
        idx = travelTime.find('hour')
        travel_1 = travelTime[:idx-1]
        travel_2 = travelTime[idx+5:-4]
        travel_1 = int(travel_1)*60
        travel_2 = int(travel_2)
        travelTime = travel_1 + travel_2
    else:
        travelTime = travelTime[:-4]
        travelTime = int(travelTime)
    return travelTime

In [8]:
#Calculates travel distance between two places
def calcTravelDist(origin, dest, mode="driving", avoid="tolls", departure_time="now"): #(can also avoid ferries)
    r = requests.get('https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + origin +
                   '&destinations=' + dest +
                   '&key=' + key +
                    '&units=imperial' +
                    '&mode=' + mode +
                    '&avoid=' + avoid +
                    '&departure_time=' + departure_time
                    ) 
    x = r.json() 
    #return x
    return(x["rows"][0]['elements'][0]['distance']['text'])

In [9]:
#Calculates travel time between two places
def calcTravelTime(origin, dest, mode="driving", avoid="tolls", departure_time="now"): #(can also avoid ferries)
    r = requests.get('https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + origin +
                   '&destinations=' + dest +
                   '&key=' + key +
                    '&units=imperial' +
                    '&mode=' + mode +
                    '&avoid=' + avoid +
                    '&departure_time=' + departure_time
                    ) 
    x = r.json()
    if (x["rows"][0]['elements'][0]['status'][0]) is str:
        print("STATUS:" + x["rows"][0]['elements'][0]['status'])
        print(x) 
        pass
    else:
        print("STATUS:" + str((x["rows"][0]['elements'][0]['status'])))
        if 'OK' in (x["rows"][0]['elements'][0]['status']):
            print("RETURNING:" + str(travelTimeInt(x["rows"][0]['elements'][0]['duration']['text'])))
            return(travelTimeInt(x["rows"][0]['elements'][0]['duration']['text']))
        else:
            print("NOT RETURNING")
            pass
    

In [10]:
#Testing of functions
#print(calcTravelDist("durango", "san francisco"))
#print(calcTravelTime("durango", "san francisco"))

<h2>Phase 2: Creating Travel Time Radius</h2>

In [11]:
#Sample map polygon
london_congestion_zone_path = gmaps.datasets.load_dataset('london_congestion_zone')
london_congestion_zone_path[:2]

[(51.530318, -0.123026), (51.530078, -0.123614)]

In [12]:
#Sample map with polygon displayed
fig = gmaps.figure(center=(51.5, -0.1), zoom_level=12) 
london_congestion_zone_polygon = gmaps.Polygon(
    london_congestion_zone_path, stroke_color='red', fill_color='blue'
)
drawing = gmaps.drawing_layer(
    features=[london_congestion_zone_polygon],
    show_controls=False )
fig.add_layer(drawing)
#fig

<ol>
<li>Ask Google for the LatLng (Latitude and Longitude) of the starting address</li>
<li>Calculate the LatLng of a set of evenly spaced destinations around the origin</li>
<li>Ask Google for the travel time to each of those destinations</li>
<li>Calculate a new distance estimate for each bearing based on this travel time</li>
<li>Repeat steps 3 and 4 for a given number of iterations</li>
<li>Pick the destinations at each bearing with the travel time closest to that entered</li>
<li>Draw this boundary on the map</li>
</ol>

In [13]:
#Converts address to lat/lng
def getGeoCodes(address):
    r = requests.get('https://maps.googleapis.com/maps/api/geocode/json?' +
                       '&address=' + address +
                       '&key=' + key
                        ) 
    x = r.json() 
    return(x['results'][0]['geometry']['location']['lat'], x['results'][0]['geometry']['location']['lng'])

In [14]:
#Testing function
#address = '1600 Amphitheatre Parkway, Mountain View, CA'
#print(getGeoCodes('1600 Amphitheatre Parkway, Mountain View, CA'))

address = '500 W Emerson St, Seattle, WA'
print(getGeoCodes('kenmore, WA'))

(47.7583576, -122.2496863)


In [15]:
#Calculates the lat/lng of a place (distance) miles away from the origin
def calculateDestination(origin, distance, bearing):
    earthRadius = 3959;
    lat, lng = getGeoCodes(origin)
    lat *=  Math.pi / 180;
    lng *= Math.pi / 180;
    destLat = Math.asin( Math.sin(lat) * Math.cos(distance/earthRadius) +
                           Math.cos(lat) * Math.sin(distance/earthRadius) * 
                           Math.cos(bearing) );
    destLng = lng + Math.atan2( Math.sin(bearing) * 
                                  Math.sin(distance/earthRadius) * Math.cos(lat),
                                  Math.cos(distance/earthRadius) - Math.sin(lat) *
                                  Math.sin(destLat) );
    #return {lat: destLat * 180 / Math.pi, lng: destLng * 180 / Math.pi};
    return {destLat * 180 / Math.pi, destLng * 180 / Math.pi};

In [16]:
#Testing function
print(calculateDestination(address, 10, 0))
print(getGeoCodes(address))

{-122.36389800000002, 47.79564625807799}
(47.6509234, -122.363898)


In [17]:
#Testing function
#Different from below because toll is one way and origin/dest are flipped
print(calcTravelDist('37.566788458078015, -122.08408970000002', '37.4220656, -122.0840897'))
print(calcTravelTime('37.566788458078015, -122.08408970000002', '37.4220656, -122.0840897'))

26.8 mi
STATUS:OK
RETURNING:33
33


In [18]:
#Converts lat/lng to format that functions for calculating distance/travel time can understand
def convertGeoCodes(coords):
    coords = str(coords)
    coords = coords[1:-1]
    return coords

In [19]:
#Testing function
print(convertGeoCodes(getGeoCodes(address)))
print(convertGeoCodes(calculateDestination(address, 10, 0)))

47.6509234, -122.363898
-122.36389800000002, 47.79564625807799


In [20]:
#Testing function
origin = convertGeoCodes(getGeoCodes(address))
dest = convertGeoCodes(calculateDestination(address, 10, 0))
print(calcTravelDist(origin, dest))
print(calcTravelTime(origin, dest))

KeyError: 'distance'

In [ ]:
#Testing function
print(travelTimeInt((calcTravelTime(origin, dest))))
print(travelTimeInt('16 hours 41 mins'))

In [ ]:
#Finds accurate point within travel time
distance = 10
timeRange = 30
bearing = 0
coords = calculateDestination(origin, distance, bearing)
coords = convertGeoCodes(coords)
travelTime = calcTravelTime(origin, coords)
travelTime = travelTimeInt(travelTime)
for i in range(10):
    if travelTime > timeRange:
        distance *= 0.9
        coords = calculateDestination(origin, distance, bearing)
        coords = convertGeoCodes(coords)
        travelTime = calcTravelTime(origin, coords)
        travelTime = travelTimeInt(travelTime)
    if travelTime < timeRange:
        distance /= 0.85
        coords = calculateDestination(origin, distance, bearing)
        coords = convertGeoCodes(coords)
        travelTime = calcTravelTime(origin, coords)
        travelTime = travelTimeInt(travelTime)

In [ ]:
#Finds accurate point within travel time
def refinePoint(origin, bearing=0, distance=5, timeRange=20):
    coords = calculateDestination(origin, distance, bearing)
    coords = convertGeoCodes(calculateDestination(origin, distance, bearing))
    print(origin)
    print(coords)
    travelTime = calcTravelTime(origin, coords) #Error spot
    print(travelTime)
    travelTime = travelTimeInt(calcTravelTime(origin, coords))
    times = []
    counter = 0
    if type(travelTime) is not int:
        print("NOT INT: " + str(type(travelTime)))
        pass
    else:
        while((travelTime > timeRange+10) or (travelTime < timeRange-10)):
            if counter > 10:
                print("COUNTER")
                counter = 0
                pass
            elif travelTime > timeRange:
                distance *= (0.9)
                coords = calculateDestination(origin, distance, bearing)
                coords = convertGeoCodes(coords)
                travelTime = calcTravelTime(origin, coords)
                travelTime = travelTimeInt(travelTime)
                counter += 1
            elif travelTime < timeRange:
                distance /= (0.85)
                coords = calculateDestination(origin, distance, bearing)
                coords = convertGeoCodes(coords)
                travelTime = calcTravelTime(origin, coords)
                travelTime = travelTimeInt(travelTime)
                #Appending times to see different options it works through
                times.append(travelTime)
                counter += 1
        travelDist = calcTravelDist(origin, coords)
        #print("TRAVEL TIME:" + travelTime)
        print("COUNTER COUNT: " + str(counter))
        
        #print(travelDist)
        return coords

In [ ]:
#Testing function
print(refinePoint(address)) #FLAGGED FOR ERROR

In [ ]:
print(getGeoCodes('XXXX'))

In [ ]:
print(calcTravelTime('32.682901110000, -108.0170992', '31.75717582903902, -110.01809920000002'))

In [ ]:
#Finds points in a circle to create drive time polygon (starts at 0 and finds nearest point every 90 degrees)
def getCircle(address):
    print(address)
    circle = []
    for i in range(0, 360, 15):
        circle.append(refinePoint(address, i))
        print(circle)
    return circle

In [ ]:
circle = getCircle('350 S Garey Ave, Pomona, CA 91766')
print(circle)

In [ ]:
#Converts polygon coordinates to float form and places them in list
def convertToPairs(circle):
    newCoords = []
    for coord in circle:
        if type(coord) is not str:
            print("Can't convert:" + str(type(coord)))
            pass
        else:
            #print(str(type(coord))) DEBUGGING
            idx = coord.find(',')
            coordPair = (float(coord[:idx]), float(coord[idx+2:]))
            #coordPair.push_back(lat)
            #coordPair.push_back(lng)
            newCoords.append(coordPair)
    return(newCoords)

In [ ]:
#Function testing
print(convertToPairs(circle))

In [ ]:
#Plot coords
def plotCoords(circle):
    fig = gmaps.figure()
    locations = convertToPairs(circle)
    symbols = gmaps.symbol_layer(
            locations, fill_color='red', stroke_color='red')
    fig.add_layer(symbols)
    return fig

In [ ]:
#Testing function
#plotCoords(circle)

In [ ]:
def plotArea(address, circle):
    coords = convertToPairs(circle)
    fig = gmaps.figure(center=getGeoCodes(address), zoom_level=11)
    drawing = gmaps.drawing_layer(features=[
         gmaps.Polygon(
             coords,
             fill_color='red',
         )
    ],
                                 show_controls=False)
    fig.add_layer(drawing)
    return fig

In [ ]:
address = 'XXXX'
circle = getCircle(address)
plotArea(address, circle)